In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bank marketing.csv


In [63]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
import numpy as np
import h5py
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [64]:
!pip install xcs

In [65]:
import xcs
from xcs import XCSAlgorithm
from xcs.scenarios import MUXProblem, ScenarioObserver
import logging
logging.root.setLevel(logging.INFO)
import random
from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

In [66]:
import pandas as pd
df = pd.read_csv("../input/bank marketing.csv")
df.head(5)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,market?
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


In [67]:
df["age"] = pd.cut(df["age"],2,labels=[0,1])
df["day"] = pd.cut(df["day"],3,labels=[0,1,2])
df["duration"] = pd.cut(df["duration"],2,labels=[0,1])
#df["education"] = pd.cut(df["education"],3,labels=[0,1,2])
df["balance"] = pd.cut(df["balance"],2,labels=[0,1])
df["housing"] = pd.cut(df["housing"],2,labels=[0,1])
df["loan"] = pd.cut(df["loan"],2,labels=[0,1])




In [68]:
df1 = pd.get_dummies(df["age"], prefix = 'age')
df2 = pd.get_dummies(df["day"], prefix = 'day')
df3 = pd.get_dummies(df["balance"], prefix = 'balance')
df4 = pd.get_dummies(df["housing"], prefix = 'housing')
df5 = pd.get_dummies(df["loan"], prefix = 'loan')
df6 = pd.get_dummies(df["duration"], prefix = 'duration')
#df7 = pd.get_dummies(df_new["education"], prefix = 'education')


In [69]:
BankDataFrame = pd.concat([df["market?"],df1,df3,df4,df5], axis=1)
BankDataFrame.head(10)

,market?,age_0,age_1,balance_0,balance_1,housing_0,housing_1,loan_0,loan_1
0,0,1,0,1,0,1,0,1,0
1,0,1,0,1,0,0,1,0,1
2,0,1,0,1,0,0,1,1,0
3,0,1,0,1,0,0,1,0,1
4,0,0,1,1,0,0,1,1,0
5,0,1,0,1,0,1,0,1,0
6,0,1,0,1,0,0,1,1,0
7,0,1,0,1,0,0,1,1,0
8,0,1,0,1,0,0,1,1,0
9,0,1,0,1,0,0,1,0,1


In [70]:
print(BankDataFrame.shape)

(4521, 9)


In [71]:
pid_dataset = BankDataFrame.values
arr = []
for i in range(len(pid_dataset)):
    arr.append(''.join([str(x) for x in pid_dataset[i]]))

In [72]:
class BankDataset(Scenario):
    def __init__(self, training_cycles=1000, input_size=200, inputs=[]):
        self.input_size = input_size
        self.possible_actions = (1, 0)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None
        self.inputs = inputs
    
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle_index = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        random.seed()
        i = random.randint(0,len(self.inputs) - 1)
        c_feat = BitString(self.inputs[i][:-1])
        self.needle_value = self.inputs[i][-1]
        return c_feat
        
    def execute(self, action):
        self.remaining_cycles -= 1
        return int(action) == int(self.needle_value)

In [73]:
inp = len(arr[0])-1
problem = BankDataset(training_cycles=10000, input_size=inp, inputs = arr)

algorithm = xcs.XCSAlgorithm()

# Default parameter settings in test()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

# Modified parameter settings
algorithm.ga_threshold = 1
algorithm.crossover_probability = .5
algorithm.wildcard_probability = .5
algorithm.deletion_threshold = 2 
algorithm.subsumption_threshold = 10         # theta_sub
algorithm.mutation_probability = .03

In [74]:
model = algorithm.new_model(ScenarioObserver(problem))
history = model.run(ScenarioObserver(problem), learn=True)

In [75]:
print(model)

######## => 0
    Time Stamp: 10000
    Average Reward: 1.0
    Error: 0.0
    Fitness: 0.023304856225485924
    Experience: 12
    Action Set Size: 94.42571650520833
    Numerosity: 1
####0##1 => 1
    Time Stamp: 9975
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.06070344482366324
    Experience: 377
    Action Set Size: 29.443575770108176
    Numerosity: 1
######1# => 0
    Time Stamp: 10000
    Average Reward: 1.0
    Error: 0.0
    Fitness: 0.08586921570938265
    Experience: 1
    Action Set Size: 96.0
    Numerosity: 1
####0### => 1
    Time Stamp: 9999
    Average Reward: 0.8751900938197714
    Error: 0.22262320349917006
    Fitness: 0.005986952323398679
    Experience: 19
    Action Set Size: 55.19960110498962
    Numerosity: 2
#1#####0 => 0
    Time Stamp: 9899
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.33540851128714244
    Experience: 66
    Action Set Size: 17.243254301296208
    Numerosity: 4
#######0 => 0
    Time Stamp: 9931
    Average Reward: 0.0
  

9


In [80]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 25:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)  

#######1 => 0  [0.98936]
#######1 => 1  [0.93909]
#######0 => 1  [0.99744]
#######0 => 0  [0.73101]
